<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/generating_scenarios.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Generate synthetic data to use in evaluations

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp; 

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the API settings by clicking the button under **"1. Create API Token"**. You can skip all other steps.

4. Add your generated API token to the cell below. 👇


In [ ]:
OKAREO_API_KEY = "<YOUR-API-KEY>"

In [ ]:
%pip install okareo 

**Example 1: Create Scenario based on seed data passed as arguments**

In [ ]:
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, SeedData, ScenarioType

okareo = Okareo(OKAREO_API_KEY)

# Generate example scenario based on seed data and return results in one call
scenario_set_create = ScenarioSetCreate(name="My Test Scenario Set ",
                                        number_examples=2,
                                        seed_data=[SeedData(input_="example question or statement", result="example result"),
                                                   SeedData(input_="tell me about your capability", result="example result"),
                                                   SeedData(input_="what are your limitations", result="example result")])

source_scenario = okareo.create_scenario_set(scenario_set_create)

print(source_scenario.app_link)

**Example 2: Create scenario set from a jsonl file upload with defined input and result fields**

In [ ]:
import os
import tempfile
import json


# Step A - Create quick sample seed data file 
# Sample seed data jsonl file for classification model, need input and result fields at minimum
rows = [
    {
        "input": "I have some quality concerns with your product, who can I talk to?",
        "result": "complaints"
    },
    {
        "input": "The product is not working as expected, I'd like to return it?",
        "result": "returns",

    },
    {
        "input": "I'd like to purchase additional filters for my model, how much are they?",
        "result": "pricing",
    }
]

temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "seed_data_sample.jsonl")

# Write to a .jsonl file
with open(file_path, "w+") as file:
    for row in rows:
        file.write(json.dumps(row) + '\n')
    

# Create scenario set with seed data file
source_scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name="Sample Scenario Upload")
print(source_scenario.app_link)

# make sure to clean up tmp file
os.remove(file_path)

**Example 3: Generate a new Scenario Set by referring to another scenario set as seed data**

In [ ]:
# Use scenario set id or scenario set object from previous step as source for generation
generated_scenario = okareo.generate_scenarios(
    source_scenario=source_scenario,
    name="Generated Scenario Set",
    number_examples=2,
)

print(generated_scenario.app_link)

**Example 4: Generate a new Scenario Set using the Text Reverse Question generator. This will generate a new scenario with questions where the original input is the answer to the question generated.**

In [ ]:
webbizz_articles = os.popen('curl https://raw.githubusercontent.com/okareo-ai/okareo-python-sdk/main/examples/webbizz_10_articles.jsonl').read()
temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "webbizz_10_articles.jsonl")
with open(file_path, "w+") as file:
    file.write(webbizz_articles)

source_scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name="Retrieval Articles Scenario")
generated_scenario_response = okareo.generate_scenarios(
    source_scenario=source_scenario,
    name="Questions from retrieval articles",
    number_examples=2,
    generation_type=ScenarioType.TEXT_REVERSE_QUESTION
)

print(generated_scenario_response.app_link)

# make sure to clean up tmp file
os.remove(file_path)